## 매수추천 지표  
- PBR(주가순자산비율) : 동종업계 평균보다 높을 시 과열이 진정되면 하락할 가능성 다분.
- PER(주가수익비율) : 작으면 기업이 내는 순이익에 비해 저평가, 크면 주가가 고평가되어있으므로 과열상태를 의미

In [ ]:
# -----------------------------------------------------------------
#    Author: 배영현                           
#    Created: 2021.06.18
#    Description: 
#    네이버 finance창의 정보를 DataFrame으로 가져온다.
#   User가 입력한 종목과 동종업계 평균의 PBR, PER을 비교한다.
#   매수추천 여부를 출력하여 투자를 돕는다.
# -----------------------------------------------------------------

In [29]:
import urllib.parse
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import time
import datetime
import math
import FinanceDataReader as web
from datetime import date, timedelta
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import numpy as np

In [30]:
## 입력받은 데이터의 종목 코드를 확인
MARKET_CODE_DICT = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt',
    'konex': 'konexMkt'
}
DOWNLOAD_URL = 'https://kind.krx.co.kr/corpgeneral/corpList.do'
MARKET_CODE_DICT

{'kospi': 'stockMkt', 'kosdaq': 'kosdaqMkt', 'konex': 'konexMkt'}

In [31]:
##종목 코드에 맞춰서 zfill=자릿수만큼 앞에 0으로 채워주는 함수

def zeroFill(columnValue):
    columnValue = str(columnValue)
    outValue = columnValue.zfill(6)
    return outValue

In [32]:
## 상장폐지가 아닌 상장사들만 조회하려는 코드이다!
## input되는 key값에 맞는 value 값 전체가 return 됨
# 코스피 종목을 불러올 함수이다.

def get_stock_list(market=None, delisted=False):
    params = {'method': 'download'}

    if market.lower() in MARKET_CODE_DICT:
        ## marketType 키 추가
        params['marketType'] = MARKET_CODE_DICT[market]
        print(market.lower()+" market key is exist")

    else:
        print("invalid market")
        
    # make url  key=value & key = value

    params_string = urllib.parse.urlencode(params)
    request_url = DOWNLOAD_URL+"?"+params_string
    df = pd.read_html(request_url)[0]
    df["종목코드"] = df.종목코드.apply(zeroFill) 
    # df["종목코드"] = df.종목코드.map('{:06d}'.format) # 동일 결과
    return df

In [33]:
inputName = input("원하는 종목명 입력")

원하는 종목명 입력 삼성전자


In [34]:
stocks = get_stock_list('kospi')

# 종목명으로 코드검색
myStockNumber = stocks.loc[stocks['회사명'] == inputName]["종목코드"].iloc[0]
myStockNumber

kospi market key is exist


'005930'

In [35]:
## 유저가 입력한 주식종목의 코드 가져오기
def get_stock_number(stock_Name):
    stocks = stock_list('kospi')
    stocksRow = stocks.loc[stocks.회사명==stock_Name]
    code = stocksRow.iloc[0]['종목코드']
    return code

In [39]:
# 네이버 금융에 들어가버리기
from bs4 import BeautifulSoup 
import requests 
import pandas as pd

url = 'https://finance.naver.com/item/main.nhn?code=' + myStockNumber
table_df_list = pd.read_html(url, encoding='euc-kr')    # 한글이 깨짐. utf-8도 깨짐. 그래서 'euc-kr'로 설정함  

earningInfo = table_df_list[3]
earningInfo

주요재무정보    최근 연간 실적                                       최근 분기 실적  \
       주요재무정보     2018.12     2019.12     2020.12  2021.12(E)    2020.03   
       주요재무정보      IFRS연결      IFRS연결      IFRS연결      IFRS연결     IFRS연결   
0         매출액  2437714.00  2304009.00  2368070.00  2675408.00  553252.00   
1        영업이익   588867.00   277685.00   359939.00   491970.00   64473.00   
2       당기순이익   443449.00   217389.00   264078.00   370036.00   48849.00   
3       영업이익률       24.16       12.05       15.20       18.39      11.65   
4        순이익률       18.19        9.44       11.15       13.83       8.83   
5   ROE(지배주주)       19.63        8.69        9.98       13.25       8.45   
6        부채비율       36.97       34.12       37.07         NaN      34.19   
7        당좌비율      204.12      233.57      214.82         NaN     237.80   
8         유보율    27531.92    28856.02    30692.79         NaN   29134.12   
9      EPS(원)     6024.00     3166.00     3841.00     5395.00     720.00   
10     PER(배)        6.42       17.63       21.09       14.99      15.24   
11     BPS(원)    35342.00    37528.00    39406.00    42038.00   38053.00   
12     PBR(배)        1.09        1.49        2.06        1.92       1.25   
13   주당배당금(원)     1416.00     1416.00     2994.00     1679.00        NaN   
14   시가배당률(%)        3.66        2.54        3.70         NaN        NaN   
15    배당성향(%)       21.92       44.73       77.95         NaN        NaN   

                                                           
      2020.06    2020.09    2020.12    2021.03 2021.06(E)  
       IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결  
0   529661.00  669642.00  615515.00  653885.00  616756.00  
1    81463.00  123532.00   90470.00   93829.00  105425.00  
2    55551.00   93607.00   66071.00   71417.00   79966.00  
3       15.38      18.45      14.70      14.35      17.09  
4       10.49      13.98      10.73      10.92      12.97  
5        8.49       9.51       9.98      10.79        NaN  
6       32.67      36.09      37.07      43.23        NaN  
7      250.04     229.69     214.82     192.26        NaN  
8    29477.97   30242.29   30692.79   30135.47        NaN  
9      808.00    1364.00     949.00    1044.00    1143.00  
10      16.52      15.89      21.09      19.54      70.78  
11   38534.00   39446.00   39406.00   39126.00        NaN  
12       1.37       1.48       2.06       2.08        NaN  
13        NaN        NaN        NaN        NaN        NaN  
14        NaN        NaN        NaN        NaN        NaN  
15        NaN        NaN        NaN        NaN        NaN

In [40]:
# NaN값 0으로 채움
earningInfo = earningInfo.fillna(0)
earningInfo

주요재무정보    최근 연간 실적                                       최근 분기 실적  \
       주요재무정보     2018.12     2019.12     2020.12  2021.12(E)    2020.03   
       주요재무정보      IFRS연결      IFRS연결      IFRS연결      IFRS연결     IFRS연결   
0         매출액  2437714.00  2304009.00  2368070.00  2675408.00  553252.00   
1        영업이익   588867.00   277685.00   359939.00   491970.00   64473.00   
2       당기순이익   443449.00   217389.00   264078.00   370036.00   48849.00   
3       영업이익률       24.16       12.05       15.20       18.39      11.65   
4        순이익률       18.19        9.44       11.15       13.83       8.83   
5   ROE(지배주주)       19.63        8.69        9.98       13.25       8.45   
6        부채비율       36.97       34.12       37.07        0.00      34.19   
7        당좌비율      204.12      233.57      214.82        0.00     237.80   
8         유보율    27531.92    28856.02    30692.79        0.00   29134.12   
9      EPS(원)     6024.00     3166.00     3841.00     5395.00     720.00   
10     PER(배)        6.42       17.63       21.09       14.99      15.24   
11     BPS(원)    35342.00    37528.00    39406.00    42038.00   38053.00   
12     PBR(배)        1.09        1.49        2.06        1.92       1.25   
13   주당배당금(원)     1416.00     1416.00     2994.00     1679.00       0.00   
14   시가배당률(%)        3.66        2.54        3.70        0.00       0.00   
15    배당성향(%)       21.92       44.73       77.95        0.00       0.00   

                                                           
      2020.06    2020.09    2020.12    2021.03 2021.06(E)  
       IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결  
0   529661.00  669642.00  615515.00  653885.00  616756.00  
1    81463.00  123532.00   90470.00   93829.00  105425.00  
2    55551.00   93607.00   66071.00   71417.00   79966.00  
3       15.38      18.45      14.70      14.35      17.09  
4       10.49      13.98      10.73      10.92      12.97  
5        8.49       9.51       9.98      10.79       0.00  
6       32.67      36.09      37.07      43.23       0.00  
7      250.04     229.69     214.82     192.26       0.00  
8    29477.97   30242.29   30692.79   30135.47       0.00  
9      808.00    1364.00     949.00    1044.00    1143.00  
10      16.52      15.89      21.09      19.54      70.78  
11   38534.00   39446.00   39406.00   39126.00       0.00  
12       1.37       1.48       2.06       2.08       0.00  
13       0.00       0.00       0.00       0.00       0.00  
14       0.00       0.00       0.00       0.00       0.00  
15       0.00       0.00       0.00       0.00       0.00

In [41]:
# 타겟주식 제일 최근의 PBR값 가져옴
targetRecntPBR = earningInfo.iloc[12,9]
targetRecntPBR

2.08

In [42]:
# 동종업계 정보표시 df
sameBusinessInfo = table_df_list[4]
sameBusinessInfo

,종목명,삼성전자*005930,SK하이닉스*000660,솔브레인*357780,리노공업058470,DB하이텍*000990
0,현재가,80700,125000,330700,170600,56700
1,전일대비,하향 200,"하향 1,500",하향 500,"상향 3,600","상향 1,100"
2,등락률,하향 -0.25%,하향 -1.19%,하향 -0.15%,상향 +2.16%,상향 +1.98%
3,시가총액(억),4817614,910002,25723,26003,25173
4,외국인비율(%),53.82,49.58,27.66,48.18,14.34
5,매출액(억),653885,84942,2392,682,2437
6,영업이익(억),93829,13244,505,279,606
7,조정영업이익(억),93829,13244,505,279,606
8,영업이익증가율(%),3.71,37.12,1.65,68.40,99.29
9,당기순이익(억),71417,9926,421,245,478


In [43]:
# 타겟주식을 제외한 동종업계 종목들 PBR 평균구하기
samebusinessPBR = sameBusinessInfo.iloc[13][2:].astype(float).mean()
samebusinessPBR

4.2875

In [44]:
goodstockCnt = 0;
if targetRecntPBR <= samebusinessPBR:
    goodstockCnt = goodstockCnt + 1;

In [45]:
# 타겟 주식 PER (2020.03 ~ 2021.03)
targetStockAvgPER = float(earningInfo.values[10][5:-1].mean())

# 동일업종 PER 나와있는 부분 df로 저장
sameBusniessPER = table_df_list[9]

# 동일업종 PER 값 글자나누고 float으로 바꿈
targetBusniessPER = sameBusniessPER[1].values[0]
targetBusniessPER = float(targetBusniessPER.split("배")[0])
targetBusniessPER


if targetStockAvgPER <= targetBusniessPER:
    goodstockCnt = goodstockCnt + 1;

In [47]:
if goodstockCnt == 0:
    print("돈을 잃고 싶다면야 말리진 않겠습니다...")
elif goodstockCnt == 1:
    print("충분히 걸만한 투자입니다")
elif goodstockCnt == 2:
    print("투자하십시오! 다만 책임은 지지 않습니다")

충분히 걸만한 투자입니다
